# [CS303]MCTS(Bonus)
For this practice, work on the `TODO` sections in the provided code to make the program run correctly.

                                                                                                      
### DDL: Oct.26
* The practice will be checked in this lab class or the next lab class before the DDL by teachers or SAs.

#### What will be tested: 
* That you understand every line of your code, not just copy from somewhere 
* That your program compiles correctly
* Correctness of the program logic 
* That the result is obtained in a reasonable time 

#### Grading: 
* Submissions on time: 30-50 points.
* Late submissions  : 0 points. 

In [8]:
from game import TicTacToe
import random
import math
import time
infinity = math.inf

## Getting Started with the Game

In [9]:
# initializing a TicTacToe game
game = TicTacToe(height=3, width=3, k=3)
board = game.initial
# get the possible actions
actions = game.actions(board)
# to apply a move (the board automatically figures out which player)
board_after_move = game.result(board, list(actions)[0])

print(board)
print(actions)
print(board_after_move)

. . .
. . .
. . .

{(0, 1), (1, 2), (2, 1), (0, 0), (1, 1), (2, 0), (0, 2), (2, 2), (1, 0)}
. . .
X . .
. . .



In [10]:
def play_game_step(game, state, strategies: dict, verbose=False):
    start = time.perf_counter()
    player = state.to_move
    move = strategies[player](game, state)
    state = game.result(state, move)
    time_elapsed = time.perf_counter() - start
    if verbose: 
        print('Player', player, 'move:', move, f'time: {time_elapsed:.4f}s', )
        print(state)
    return state

def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    state = game.initial
    while not game.is_terminal(state):
        state = play_game_step(game, state, strategies, verbose)
    return state

# setup a random strategy for testing
def random_player(game, state): return random.choice(list(game.actions(state)))

def search_player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)[1] # we expect our search algorithm to return (v, move)

In [11]:
# initialize a game
game = TicTacToe()
state = game.initial
strategies = dict(X=random_player, O=random_player)

In [12]:
# option 1: 
play_game(game, strategies, True)

# option 2: run it step by step by executing this cell multiple times until the game end
# if not game.is_terminal(state):
#     state = play_game_step(game, state, strategies, True)
# else:
#     print(f"Game result: {game.utility(state, 'X')}")
#     print(state)

Player X move: (2, 0) time: 0.0000s
. . X
. . .
. . .

Player O move: (1, 0) time: 0.0000s
. O X
. . .
. . .

Player X move: (1, 2) time: 0.0000s
. O X
. . .
. X .

Player O move: (2, 1) time: 0.0000s
. O X
. . O
. X .

Player X move: (2, 2) time: 0.0000s
. O X
. . O
. X X

Player O move: (1, 1) time: 0.0000s
. O X
. O O
. X X

Player X move: (0, 1) time: 0.0000s
. O X
X O O
. X X

Player O move: (0, 0) time: 0.0000s
O O X
X O O
. X X

Player X move: (0, 2) time: 0.0000s
O O X
X O O
X X X



O O X
X O O
X X X

## MONTE-CARLO-TREE-SEARCH

__function__ MONTE-CARLO-TREE-SEARCH(_state_) __returns__ an action  <br>
&emsp;tree &larr; NODE(_state_)       <br>
&emsp;__while__ TIME\-REMAINING() __do__  <br>
&emsp;&emsp;&emsp;__tree__ &larr; PLAYOUT(_tree_) <br>
&emsp;__return__ the _move_ in ACTIONS(_state_) with highest Q(_state_,_move_)  

---

__function__ PLAYOUT(_tree_) __returns__ _updated tree_  <br>
&emsp;_node_ &larr; _tree_  <br>
&emsp;__while__ _node_ is not terminal and was already in _tree_ __do__  <br>
&emsp;&emsp;&emsp;_move_ &larr; SELECT(_node_) <br>
&emsp;&emsp;&emsp;_node_ &larr; FOLLOW\-LINK(_node_,_move_)  <br>
&emsp;_outcome_ &larr; SIMULATION(_node_.STATE) <br> 
&emsp;UPDATE(_node_,_outcome_)  <br>
&emsp;__return__ _tree_  

---

__function__ SELECT(_node_) __returns__ _an action_  <br>
&emsp;__return__ argmax<sub>m &isin; FEASIBLE\-ACTIONS(_node_)</sub> UCB(RESULT(_node_,_m_))  

---

__function__ UCB(_child_) __returns__ _a number_  <br>
&emsp;__return__ _child_.VALUE + C &times; <a href="https://www.codecogs.com/eqnedit.php?latex=\inline&space;\sqrt{\frac{\log{child.PARENT.N}}{child.N}}" target="_blank"><img src="https://latex.codecogs.com/png.latex?\inline&space;\sqrt{\frac{\log{child.PARENT.N}}{child.N}}" title="\sqrt{\frac{\log{child.PARENT.N}}{child.N}}" /></a> 

---

The Monte Carlo tree search algorithm. A game tree, _tree_, is initialized, and then grows by one node with each call to PLAYOUT. The function SELECT chooses a move that best balances exploitation and exploration according to the UCB formula. FOLLOW-LINK traverses from the current node by making a move; this could be to a previously-seen node, or to a new node that is added to the tree. Once we have added a new node, we exit the __while__ loop and SIMULATION chooses moves (with a randomized policy that is designed to favor good moves but to compute quickly) until the game is over. Then, UPDATE updates all the nodes in the tree from node to the root, recording the fact that the path led to the final __outcome__.

In [20]:
import numpy as np
class MCT_Node:
    """Node in the Monte Carlo search tree, keeps track of the children states."""

    def __init__(self, parent=None, state=None, U=0, N=0):
        self.__dict__.update(parent=parent, state=state, U=U, N=N)
        self.children = {} # dict of {node: action} or {actione: node}
        self.actions = None

def ucb(n, C=1.4):
    return np.inf if n.N == 0 else n.U / n.N + C * np.sqrt(np.log(n.parent.N) / n.N)

def monte_carlo_tree_search(game, state, N=1000):
    def select(n):
        """
        select a leaf node in the tree according to their UCB values
        """
        if n.children:
            return select(max(n.children.keys(), key=ucb))
        else:
            return n

    def expand(n):
        """ 
        expand the leaf node by adding all its children states (FOLLOW-LINK)
        """
        if not n.children and not game.is_terminal(n.state):
            n.children = {MCT_Node(parent=n, state=game.result(n.state, action)) : action for action in game.actions(n.state)} # TODO: fix this
        return select(n)

    def simulate(game, state):
        """ 
        simulate the utility of current state using a random strategy
        """
        player = state.to_move
        while not game.is_terminal(state):
            # TODO: fix and complete this
            action = random.choice(list(game.actions(state)))
            state = game.result(state, action)

        v = game.utility(state, player)
        return v # utility 是从该玩家视角出发

    def backprop(n, utility):
        """
        passing the utility back to all parent nodes
        """
        if utility > 0:
            n.U += utility #赢的次数
        n.N += 1
        if n.parent:
            backprop(n.parent, -utility) #零和，传递负值

    root = MCT_Node(state=state)

    for _ in range(N):
        # PLAYOUT
        leaf = select(root)
        child = expand(leaf)
        result = simulate(game, child.state)
        backprop(child, result)
    
    # TODO: select the action
    best_child = min(root.children.keys(), key=lambda n: n.U / n.N)
    move = root.children[best_child]
    v = best_child.U / best_child.N if best_child.N > 0 else 0
    return v, move

# if you have done it right it should usually win
%time play_game(game, {'X':search_player(monte_carlo_tree_search), 'O':random_player}, True)

Player X move: (0, 2) time: 0.0717s
. . .
. . .
X . .

Player O move: (0, 0) time: 0.0000s
O . .
. . .
X . .

Player X move: (2, 2) time: 0.0258s
O . .
. . .
X . X

Player O move: (0, 1) time: 0.0000s
O . .
O . .
X . X

Player X move: (1, 2) time: 0.0089s
O . .
O . .
X X X

CPU times: user 105 ms, sys: 1.48 ms, total: 107 ms
Wall time: 107 ms


O . .
O . .
X X X